In [7]:
#installations
!pip install torchaudio
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [8]:
#imports
from itertools import product
import os
import music21
import pretty_midi
from music21 import midi
import pandas as pd
from IPython.display import Audio
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics
import math, random, time
from IPython.display import Audio
from google.colab import drive
from google.colab import files
import torchaudio
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import scipy
import shutil

In [9]:
#mount drive, establish data directory
from google.colab import drive
drive.mount('/content/drive')
data_directory = '/content/drive/MyDrive/maestro-v2.0.0-csv/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# define midi reading function
def readMidi(filepath):
  mf = midi.MidiFile()
  mf.open(filepath)
  mf.read()
  mf.close()
  return mf

In [5]:
import os
import shutil
from tqdm import tqdm
import pretty_midi
import numpy as np
import torch
from torch.utils.data import Dataset

class MaestroDataset(Dataset):
    def __init__(self, split, token_dict, directory='/content/drive/MyDrive/maestro-v2.0.0-csv/'):
        import pandas as pd

        self.split = split
        self.token_dict = token_dict
        self.directory = directory.rstrip('/') + '/'
        # read the CSV and filter to our split
        df = pd.read_csv(os.path.join(self.directory, 'maestro-v2.0.0.csv'))
        self.df = df[df['split'] == split].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def moveFilesToSplitDir(self):
        src_root = self.directory
        dst_root = os.path.join(self.directory, self.split)
        os.makedirs(dst_root, exist_ok=True)

        for fname in self.df['midi_filename']:
            src = os.path.join(src_root, fname)
            if not os.path.isfile(src):
                print(f"Warning: MIDI file not found: {src}")
                continue
            dst = os.path.join(dst_root, os.path.basename(fname))
            shutil.move(src, dst)

    def create_token_sequences(self,
                               pm: pretty_midi.PrettyMIDI,
                               fs: int = 16,
                               seq_len: int = 500,
                               add_bos_eos: bool = True):
        pianoroll = pm.get_piano_roll(fs=fs)
        pianoroll = (pianoroll > 0).astype(int)

        num_frames = pianoroll.shape[1]
        seqs = []
        for start in range(0, num_frames, seq_len):
            end = start + seq_len
            if end > num_frames:
                break
            block = pianoroll[:, start:end]
            # skip mostly-silent windows
            if block.sum() == 0:
                continue

            seq = []
            if add_bos_eos:
                seq.append(self.token_dict['<bos>'])
            for t in range(seq_len):
                notes = np.nonzero(block[:, t])[0]
                if len(notes):
                    seq.append(int(notes[0]))
                else:
                    seq.append(self.token_dict['<rest>'])
            if add_bos_eos:
                seq.append(self.token_dict['<eos>'])
            seqs.append(seq)
        return seqs

    def augment_idxs(self, seq):
        augmented_shift = []
        for shift in range(-12, +13):
            out = []
            for idx in seq:
                # only shift real MIDI notes; leave special tokens alone
                if 0 <= idx < 128:
                    n = idx + shift
                    n = max(0, min(127, n))
                    out.append(n)
                else:
                    out.append(idx)
            augmented_shift.append((shift, out))
        return augmented_shift

    def midi_tok_augment(self, seq, output_dir=None, pitch_shifts=range(-12,13), beat_offsets=range(-4,4)):
        split_dir = os.path.join(self.directory, self.split)
        if output_dir is None:
            output_dir = os.path.join(split_dir, "augmented_midi")
        os.makedirs(output_dir, exist_ok=True)

        results = {}

        for fn in tqdm(os.listdir(split_dir)):
            if not fn.lower().endswith(('.mid', '.midi')):
                continue

            path_in = os.path.join(split_dir, fn)
            pm = pretty_midi.PrettyMIDI(path_in)
            pm.remove_invalid_notes()

            # this will be a list of PrettyMIDI objects
            aug_list = miditok.data_augmentation.augment_midi_multiple_offsets(
                pm,
                pitch_shifts=list(pitch_shifts),
                beat_offsets=list(beat_offsets),
                all_offset_combinations=True
            )

            out_paths = []
            base, ext = os.path.splitext(fn)

            # zip together the shift/offset pairs and the generated PMs
            for (ps, bo), pm_obj in zip(product(pitch_shifts, beat_offsets), aug_list):
                # name each file so you can see what transformation was applied
                out_name = f"{base}_ps{ps:+d}_bo{bo:+d}{ext}"
                path_out = os.path.join(output_dir, out_name)
                pm_obj.write(path_out)
                out_paths.append(path_out)

            results[fn] = out_paths

        return results


    def create_all_token_sequences(self,
                                   fs: int = 16,
                                   seq_len: int = 500):
        split_dir = os.path.join(self.directory, self.split)
        print("-> files in", split_dir, "=", os.listdir(split_dir)[:10])
        all_seqs = []

        for file_name in tqdm(os.listdir(split_dir)):
            if not file_name.lower().endswith(('.mid', '.midi')):
                continue
            path = os.path.join(split_dir, file_name)
            try:
                pm = pretty_midi.PrettyMIDI(path)
                pm.remove_invalid_notes()
                base_name = os.path.basename(file_name)
                compare = self.df['midi_filename'].apply(os.path.basename) == base_name
                print("    looking for", file_name, " -> matched rows:", compare.sum())
                if not compare.any():
                    print(f"Warning: MIDI file not found in CSV: {path}")
                    continue
                composer_label = self.df.loc[compare, 'canonical_composer'].values[0]
                title_label = self.df.loc[compare, 'canonical_title'].values[0]
            except Exception as e:
                print(f"Skipping {file_name}: {e}")
                continue

            seqs = self.create_token_sequences(pm, fs=fs, seq_len=seq_len)
            for s in seqs:
                all_seqs.append({'title': title_label, 'sequence': s, 'composer': composer_label, 'transposition amount': 0})
                for pshift, aug in self.augment_idxs(s):
                      all_seqs.append({'title': title_label, 'sequence': aug, 'composer': composer_label, 'transposition amount': pshift})
        return all_seqs

In [ ]:
# build token dict once
token_dict = {i:i for i in range(128)}
token_dict.update({'<rest>':128, '<bos>':129, '<eos>':130})
inverse_token_dict = {val:num for val,num in token_dict.items()}

csv_rows = []

for phase in ['train', 'test', 'validation']:
    ds = MaestroDataset(phase, token_dict,
                        directory='/content/drive/MyDrive/maestro-v2.0.0-csv/')
    ds.moveFilesToSplitDir()
    # seqs = ds.create_all_token_sequences(fs=16, seq_len=500)
    seqs = ds.create_all_token_sequences_miditok(fs=16, seq_len=500, do_midi_augment=True)

    for seq in seqs:
        seq['split'] = phase
        seq['sequence'] = ' '.join(map(str, seq['sequence']))

    csv_rows.extend(seqs)

df = pd.DataFrame(csv_rows)
dir_csv = '/content/drive/MyDrive/maestro-v2.0.0-csv/'
df.to_csv('maestro_token_sequences.csv', index=False)
print(f'Wrote {len(df)} rows to {dir_csv}')
    # ds.tokens_seqs_to_txt(seqs, phase)

-> files in /content/drive/MyDrive/maestro-v2.0.0-csv/train = ['MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_03_Track03_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_04_Track04_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_02_Track02_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_12_Track12_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_06_Track06_wav.midi', 'MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_08_Track08_wav.midi', 'MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_07_Track07_wav.midi']


  0%|          | 0/923 [00:00<?, ?it/s]

    looking for MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi  -> matched rows: 1


  0%|          | 1/923 [00:01<26:11,  1.70s/it]

    looking for MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi  -> matched rows: 1


...

    looking for MIDI-Unprocessed_23_R1_2011_MID--AUDIO_R1-D9_05_Track05_wav.midi  -> matched rows: 1


 99%|█████████▉| 130/131 [01:00<00:00,  3.08it/s]

    looking for MIDI-Unprocessed_23_R3_2011_MID--AUDIO_R3-D8_03_Track03_wav.midi  -> matched rows: 1
    looking for MIDI-Unprocessed_25_R3_2011_MID--AUDIO_R3-D9_05_Track05_wav.midi  -> matched rows: 1


100%|██████████| 131/131 [01:00<00:00,  2.18it/s]


Wrote 561340 rows to /content/drive/MyDrive/maestro-v2.0.0-csv/


In [11]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold, train_test_split

input_csv = '/content/drive/MyDrive/maestro_token_sequences.csv'

def prepare_data(file, augmentations=False, collaborations=False):
    # Read the data
    df = pd.read_csv(file)

# #only read train data
#     df = df[df['split'] == split].copy()
    # Convert sequences to vectors
    df['sequence_vector'] = df['sequence'].apply(string_to_vector)

#Apply filters
    if not collaborations:
        df = df[~df['composer'].str.contains('/', na=False)].copy()
    if not augmentations:
        df = df[df['transposition amount'] == 0].copy()

#Create labels (assuming composer_label_dict exists)
    clean_df = df.copy()  # Final cleaned version
    composer_list = sorted(list(set(clean_df['composer'])))  # Convert to sorted list for consistent ordering
    num_composers = len(composer_list)
    print(f"Unique composers: {composer_list}")
    print(f"Total composers: {num_composers}")

    # Create proper label dictionary
    composer_label_dict = {composer: idx for idx, composer in enumerate(composer_list)}
    index_composer_dict = {idx: composer for composer, idx in composer_label_dict.items()}
    clean_df['label'] = clean_df['composer'].map(composer_label_dict)

#Select only the two columns we want
    data = clean_df[['sequence_vector', 'label']].copy()

    return data,composer_label_dict,index_composer_dict

def string_to_vector(seq_str):
    return [int(x) for x in seq_str.split()]

def cross_validation_splits(data, labels, max_folds=5, random_state=None):
    #place data into different split containers
    splits = {}

    for k in range(2, max_folds + 1):
        for bool in (False, True):
            kf = KFold(n_splits=k, shuffle=bool, random_state=random_state)
            split_indices = kf.split(data, labels)
            splits[f'{k}-fold' + ('-shuffled' if bool else '')] = list(split_indices)
    return splits

def split_data(features, labels):
    X_train, X_temp, y_train, y_temp = train_test_split(
        features,
        labels,
        test_size=0.4,
        stratify=labels,
        random_state=42
    )

    X_test, X_val, y_test, y_val = train_test_split(
        X_temp,
        y_temp,
        test_size=0.25,
        stratify=y_temp,
        random_state=42
    )

    return X_train, X_test, y_train, y_test, X_val, y_val


def build_fold_data(data, labels, splits):
    fold_data = {}
    for key, idx_vals in splits.items():
        fold_data[key] = []
        for train_idx, test_idx in idx_vals:
            data_train = data.iloc[train_idx].reset_index(drop=True)
            data_test = data.iloc[test_idx].reset_index(drop=True)
            label_train = labels.iloc[train_idx].reset_index(drop=True)
            label_test = labels.iloc[test_idx].reset_index(drop=True)

            fold_data[key].append({
                'training data': data_train,
                'test data': data_test,
                'training labels': label_train,
                'test labels': label_test
            })
    return fold_data

def export_all_folds(fold_data, output_path='all_folds.csv'):
    all_rows = []
    for split, folds in fold_data.items():
        for i, fold in enumerate(folds, start=1):
            df_train = fold['training data'].copy()
            df_train['label'] = fold['training labels']
            df_train['set'] = 'train'
            df_train['split'] = split
            all_rows.append(df_train)

            df_test = fold['test data'].copy()
            df_test['label'] = fold['test labels']
            df_test['set'] = 'test'
            df_test['split'] = split
            all_rows.append(df_test)

    whole_df = pd.concat(all_rows, ignore_index=True)
    whole_df.to_csv("fold_train.csv.gz", index=False, compression="gzip")
    print(f"Wrote combined CSV to {output_path}")

data, composer_label_dict, index_composer_dict = prepare_data(input_csv)

prepared_data = data['sequence_vector']
prepared_labels = data['label']

# splits = cross_validation_splits(prepared_data, prepared_labels, max_folds=10)
# all_fold_data = build_fold_data(prepared_data, prepared_labels, splits)
# export_all_folds(all_fold_data)

data_train, data_test, labels_train, labels_test, data_val, labels_val = split_data(prepared_data, prepared_labels)

def make_df(Data, labels, split_name):
    df = pd.DataFrame({
        'sequence_vector': Data.values,
        'label':            labels.values,
    })
    df['split'] = split_name
    return df

df_train = make_df(data_train, labels_train, 'train')
df_test  = make_df(data_test , labels_test , 'test')
df_val   = make_df(data_val  , labels_val  , 'validation')

df_all = pd.concat([df_train, df_test, df_val], ignore_index=True)
# df_all.to_csv('maestro_new_splits.csv', index=False)
output_file = "maestro_new_splits_no_augmentation_403010.csv"
chunk_size = 2500


with open(output_file, 'w', newline='') as f:
    for start in tqdm(range(0, len(df_all), chunk_size), desc="Writing CSV"):
        end   = start + chunk_size
        chunk = df_all.iloc[start:end]
        chunk.to_csv(f, index=False, header=(start == 0))

KeyboardInterrupt: 